# エニグマの実装を実装

参考ページ：https://qiita.com/deaikei/items/01e962c4c15b2efcc84f

## エニグマとは？



In [1]:
import random

class Converter(object):
    def __init__(self, size, seed):
        self.size = size
        conversions = list(range(size))
        random.seed(seed)
        random.shuffle(conversions)
        self.forwards = tuple(conversions)
        self.backwards = tuple(conversions.index(port) for port in range(size))

    def forward(self, port):
        return self.forwards[port]

    def backward(self, port):
        return self.backwards[port]

class Scrumbler(Converter):
    def __init__(self, size, seed, rotation=0):
        Converter.__init__(self, size, seed)
        self.set_rotation(rotation)

    def set_rotation(self, rotation):
        self.rotation = rotation

    def rotate(self, step=1):
        rotation = self.rotation + step
        self.set_rotation(rotation % self.size)
        return rotation // self.size    # carry

    def forward(self, port):
        return self._convert(port, self.forwards)

    def backward(self, port):
        return self._convert(port, self.backwards)

    def _convert(self, port, conversions):
        port = (port + self.rotation) % self.size
        port = conversions[port]
        port = (port - self.rotation) % self.size
        return port

class Refrector(object):
    def __init__(self, size, seed):
        refrections = list(range(size))
        random.seed(seed)
        random.shuffle(refrections)
        self.refrections = tuple(refrections[-1 - refrections.index(port)] for port in range(size))

    def refrect(self, port):
        return self.refrections[port]

class Enigma:
    def __init__(self, characters, rotations=(0,0,0), seeds=(0,1,2,3,4)):
        self.letters = characters
        self.rotations = rotations
        # mechanical part
        # enigma machine is composed of three parts:
        # plug board, scrumbler and reflector
        size = len(characters)
        self.plug_board = Converter(size, seeds[0])
        self.scrumblers = tuple(Scrumbler(size, seed, rotation)
                                for seed, rotation in zip(seeds[1:-1], rotations))
        self.refrector = Refrector(size, seeds[-1])

    # initialize settings for the scrumblers
    def set_rotations(self, rotations=None):
        if rotations is None:
            rotations = self.rotations
        for scrumbler, rotation in zip(self.scrumblers, rotations):
            scrumbler.set_rotation(rotation)

    # rotate scrunbler
    def _rotate_scrumblers(self):
        for scrumbler in self.scrumblers:
            if scrumbler.rotate() == 0: # no carry
                return

    # encode a letter
    def _encode_letter(self, letter):
        port = self.letters.index(letter)
        # forward transmit
        port = self.plug_board.forward(port)
        for scrumbler in self.scrumblers:
            port = scrumbler.forward(port)
        # refrect
        port = self.refrector.refrect(port)
        # backward transmit
        for scrumbler in self.scrumblers[::-1]:
            port = scrumbler.backward(port)
        port = self.plug_board.backward(port)
        # rotate scrumblers
        self._rotate_scrumblers()
        return self.letters[port]

    # encode a message
    def encode(self, message):
        return ''.join(self._encode_letter(letter) for letter in message)

# character set
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
            'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
            'w', 'x', 'y', 'z', ' ', '?']

In [2]:
print(alphabet)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '?']


In [3]:
en = Enigma(characters=alphabet)


In [8]:
heibun = input()
angou= en.encode(heibun)
print(angou)
en.set_rotations()
print(en.encode(angou))

 gekki


ti?ck
ok ze
